# <h1 style="color:yellow"><strong>Pacotes e abertura dos dados</strong></h1>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dados/dados_ecommerce_limpos.csv', sep=';')

df.drop(columns=['Unnamed: 0'],inplace= True)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df.head()

,InvoiceNo,StoreId,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,11,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,9,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,17,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,15,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,17,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539109 entries, 0 to 539108
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    539109 non-null  object        
 1   StoreId      539109 non-null  int64         
 2   StockCode    539109 non-null  object        
 3   Description  539109 non-null  object        
 4   Quantity     539109 non-null  int64         
 5   InvoiceDate  539109 non-null  datetime64[ns]
 6   UnitPrice    539109 non-null  float64       
 7   CustomerID   406506 non-null  float64       
 8   Country      539109 non-null  object        
 9   TotalPrice   539109 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 41.1+ MB


# <h1 style="color:yellow"><strong>1) Quais as 10 lojas com maior faturamento em pedidos? Sumarize os dados dessa loja apresentando os seguintes campos:</strong></h1>
# <ul style="color:yellow">
#   <li style="color:yellow">StoreId</li>
#   <li style="color:yellow">Volume total de itens vendidos</li>
#   <li style="color:yellow">Valor total vendido</li>
# </ul>

In [4]:
faturamento_loja = df.groupby('StoreId')['Quantity', 'TotalPrice'].sum().reset_index()
faturamento_loja.sort_values(['TotalPrice'], inplace=True)
faturamento_loja = faturamento_loja.loc[::-1].head(10).reset_index(drop=True)
faturamento_loja

<ipython-input-4-338f6bf7a807>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  faturamento_loja = df.groupby('StoreId')['Quantity', 'TotalPrice'].sum().reset_index()


,StoreId,Quantity,TotalPrice
0,17,356701,725067.200
1,19,362332,721879.430
2,4,365025,688513.190
3,20,347350,613752.720
4,14,264195,578242.321
5,10,274856,572725.380
6,16,268243,571324.610
7,13,283209,570519.820
8,1,284368,561231.491
9,6,277833,560081.901


# <h1 style="color:yellow"><strong>2) Qual ticket médio mensal dos pedidos apresentados? Qual o volume médio mensal de vendas (todas as lojas)?</strong></h1>

In [41]:
pedidos = df.groupby(['InvoiceNo', 'InvoiceDate'])['TotalPrice', 'Quantity'].sum().reset_index()

ticket_medio = pedidos.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['TotalPrice'].mean().reset_index()
ticket_medio.sort_values(['TotalPrice'], inplace=True)
ticket_medio = ticket_medio.loc[::-1].reset_index(drop=True)
ticket_medio

<ipython-input-41-d219602a69f5>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pedidos = df.groupby(['InvoiceNo', 'InvoiceDate'])['TotalPrice', 'Quantity'].sum().reset_index()


,InvoiceDate,TotalPrice
0,2011-12-31,874.494642
1,2011-01-31,608.040732
2,2011-10-31,515.415997
3,2011-08-31,508.573019
4,2011-09-30,505.459351
5,2011-11-30,484.572334
6,2010-12-31,476.345175
7,2011-06-30,446.737483
8,2011-07-31,433.628781
9,2011-03-31,422.681754


In [42]:
volume_medio = pedidos.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['Quantity'].mean().reset_index()
volume_medio.sort_values(['Quantity'], inplace=True)
volume_medio = volume_medio.loc[::-1].reset_index(drop=True)
volume_medio

,InvoiceDate,Quantity
0,2011-12-31,412.239378
1,2011-01-31,345.609017
2,2011-10-31,269.534332
3,2011-09-30,266.328420
4,2011-08-31,264.211199
5,2011-11-30,238.705571
6,2011-07-31,233.546392
7,2011-04-30,216.095174
8,2011-02-28,215.820648
9,2011-03-31,215.230467


# <h1 style="color:yellow"><strong>3) Existe alguma correlação entre o país e o volume de itens vendidos? Qual o item mais vendido e o menos vendido no país com maior volume de vendas?</strong></h1>

In [6]:
vendas_pais = df.groupby('Country')['Quantity', 'TotalPrice'].sum().reset_index()
vendas_pais.sort_values(['Quantity'], inplace=True)
vendas_pais = vendas_pais.loc[::-1].reset_index(drop=True)
vendas_pais.head()

<ipython-input-6-58ca7995475b>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vendas_pais = df.groupby('Country')['Quantity', 'TotalPrice'].sum().reset_index()


,Country,Quantity,TotalPrice
0,United Kingdom,4924659,9.860952e+06
1,Netherlands,201170,2.862311e+05
2,EIRE,151983,3.036311e+05
3,Germany,121051,2.359759e+05
4,France,113727,2.220263e+05


In [26]:
pedidos_UK = df[df.Country == 'United Kingdom'].groupby(['StockCode', 'Description'])['Quantity'].sum().reset_index()
pedidos_UK.sort_values(['Quantity'], inplace=True)
pedidos_UK = pedidos_UK.loc[::-1].reset_index(drop=True)
pedidos_UK.head()

,StockCode,Description,Quantity
0,23843,"PAPER CRAFT , LITTLE BIRDIE",161990
1,23166,MEDIUM CERAMIC TOP STORAGE JAR,151503
2,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,50726
3,85099B,JUMBO BAG RED RETROSPOT,45362
4,85123A,WHITE HANGING HEART T-LIGHT HOLDER,37999


In [28]:
pedidos_UK.iloc[[0]]

,StockCode,Description,Quantity
0,23843,"PAPER CRAFT , LITTLE BIRDIE",161990


In [29]:
pedidos_UK[pedidos_UK.Quantity == 1]

,StockCode,Description,Quantity
4062,84569C,PACK 4 FLOWER/BUTTERFLY PATCHES,1
4063,21160,KEEP OUT GIRLS DOOR HANGER,1
4064,90126A,PINK BERTIE MOBILE PHONE CHARM,1
4065,79320,FLAMINGO LIGHTS,1
4066,23609,SET 10 CARDS SNOWY ROBIN 17099,1
...,...,...,...
4151,90025B,BAROQUE BUTTERFLY EARRINGS RED,1
4152,90025E,BAROQUE BUTTERFLY EARRINGS MONTANA,1
4153,84743C,ORANGE FELT VASE + FLOWERS,1
4154,90068,"RUBY GLASS NECKLACE 42""",1


# <h1 style="color:yellow"><strong>4) Extra - A partir dos números apresentados e o seu conhecimento prévio, apresente ideais/estratégias que podem ser implementadas para auxiliar nossos lojistas a aumentarem o volume de vendas?</strong></h1>